In [1]:
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

2024-01-18 12:04:28.417762: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Seperation von Featchers und Zieldaten (x, y)

from pathlib import Path
cwd = str(Path.cwd())

X = pd.read_csv(cwd + "/../data/Featchers_randomized.csv")
Y = pd.read_csv(cwd + "/../data/Goals_randomized.csv")
D = pd.read_csv(cwd + "/../data/Timings_randomized.csv").sort_index(axis=1)

D['straubing_DATE'] = pd.to_datetime(D['straubing_DATE'], format='%d.%m.%Y')

X_Val = X.values

Y_Temp_1 = Y[["straubing_LUFTTEMPERATUR_future_1Day"]].values.flatten()
Y_Temp_2 = Y[["straubing_LUFTTEMPERATUR_future_2Day"]].values.flatten()
Y_Temp_3 = Y[["straubing_LUFTTEMPERATUR_future_3Day"]].values.flatten()
Y_Reg_1 = Y[["straubing_NIEDERSCHLAGSHOEHE_future_1Day"]].values.flatten()
Y_Reg_2 = Y[["straubing_NIEDERSCHLAGSHOEHE_future_2Day"]].values.flatten()
Y_Reg_3 = Y[["straubing_NIEDERSCHLAGSHOEHE_future_3Day"]].values.flatten()

In [4]:
# Führe den Train-Test-Split für jedes DataFrame durch
X_train_Temp_1, X_test_Temp_1, y_train_Temp_1, y_test_Temp_1 = train_test_split(X_Val, Y_Temp_1, test_size=0.2)
X_train_Temp_2, X_test_Temp_2, y_train_Temp_2, y_test_Temp_2 = train_test_split(X_Val, Y_Temp_2, test_size=0.2)
X_train_Temp_3, X_test_Temp_3, y_train_Temp_3, y_test_Temp_3 = train_test_split(X_Val, Y_Temp_3, test_size=0.2)

X_train_Reg_1, X_test_Reg_1, y_train_Reg_1, y_test_Reg_1 = train_test_split(X_Val, Y_Reg_1, test_size=0.2)
X_train_Reg_2, X_test_Reg_2, y_train_Reg_2, y_test_Reg_2 = train_test_split(X_Val, Y_Reg_2, test_size=0.2)
X_train_Reg_3, X_test_Reg_3, y_train_Reg_3, y_test_Reg_3 = train_test_split(X_Val, Y_Reg_3, test_size=0.2)

In [5]:
def nn(X, Y):
    # Initialize the Neural Network
    model = Sequential()
    model.add(Dense(10, input_dim=X.shape[1], activation='relu'))  # Adjust number of neurons
    model.add(Dense(1))  # Output layer

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    model.fit(X, Y, epochs=150, batch_size=32)  # Adjust epochs and batch_size as needed
    return model


In [6]:
model_Temp_1 = nn(X_train_Temp_1, y_train_Temp_1)
model_Temp_2 = nn(X_train_Temp_2, y_train_Temp_2)
model_Temp_3 = nn(X_train_Temp_3, y_train_Temp_3)

model_Reg_1 = nn(X_train_Reg_1, y_train_Reg_1)
model_Reg_2 = nn(X_train_Reg_2, y_train_Reg_2)
model_Reg_3 = nn(X_train_Reg_3, y_train_Reg_3)

Epoch 1/150


2024-01-18 12:08:40.587538: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


158/158 [==============================] - 1s 5ms/step - loss: 123.6471
Epoch 2/150
158/158 [==============================] - 1s 6ms/step - loss: 18.3930
Epoch 3/150
158/158 [==============================] - 1s 4ms/step - loss: 11.3727
Epoch 4/150
158/158 [==============================] - 1s 7ms/step - loss: 8.6371
Epoch 5/150
158/158 [==============================] - 1s 6ms/step - loss: 7.3271
Epoch 6/150
158/158 [==============================] - 1s 6ms/step - loss: 6.5591
Epoch 7/150
158/158 [==============================] - 2s 9ms/step - loss: 5.9069
Epoch 8/150
158/158 [==============================] - 1s 6ms/step - loss: 5.6015
Epoch 9/150
158/158 [==============================] - 1s 7ms/step - loss: 5.2152
Epoch 10/150
158/158 [==============================] - 2s 11ms/step - loss: 5.0868
Epoch 11/150
158/158 [==============================] - 1s 4ms/step - loss: 4.9261
Epoch 12/150
158/158 [==============================] - 1s 7ms/step - loss: 4.7624
Epoch 13/150
158/158

In [7]:

# Predictions
predictions_Temp_1 = model_Temp_1.predict(X_test_Temp_1).flatten()
predictions_Temp_2 = model_Temp_2.predict(X_test_Temp_2).flatten()
predictions_Temp_3 = model_Temp_3.predict(X_test_Temp_3).flatten()

predictions_Reg_1 = model_Reg_1.predict(X_test_Reg_1).flatten()
predictions_Reg_2 = model_Reg_2.predict(X_test_Reg_2).flatten()
predictions_Reg_3 = model_Reg_3.predict(X_test_Reg_3).flatten()


40/40 [==============================] - 0s 8ms/step


In [16]:
from sklearn.metrics import mean_squared_error, r2_score
r2_Temp_1 = r2_score(predictions_Temp_1, y_test_Temp_1)
r2_Temp_2 = r2_score(predictions_Temp_2, y_test_Temp_2)
r2_Temp_3 = r2_score(predictions_Temp_3, y_test_Temp_3)

r2_Reg_1 = r2_score(predictions_Reg_1, y_test_Reg_1)
r2_Reg_2 = r2_score(predictions_Reg_2, y_test_Reg_2)
r2_Reg_3 = r2_score(predictions_Reg_3, y_test_Reg_3)

mse_Temp_1 = mean_squared_error(predictions_Temp_1, y_test_Temp_1)
mse_Temp_2 = mean_squared_error(predictions_Temp_2, y_test_Temp_2)
mse_Temp_3 = mean_squared_error(predictions_Temp_3, y_test_Temp_3)

mse_Reg_1 = mean_squared_error(predictions_Reg_1, y_test_Reg_1)
mse_Reg_2 = mean_squared_error(predictions_Reg_2, y_test_Reg_2)
mse_Reg_3 = mean_squared_error(predictions_Reg_3, y_test_Reg_3)

In [17]:
print(r2_Temp_1)
print(r2_Temp_2)
print(r2_Temp_3)

print(r2_Reg_1)
print(r2_Reg_2)
print(r2_Reg_3)

print(mse_Temp_1)
print(mse_Temp_2)
print(mse_Temp_3)

print(mse_Reg_1)
print(mse_Reg_2)
print(mse_Reg_3)

0.9277938056555194
0.8359290540674778
0.7763533941336647
-7.519439317629894
-4.137395946775079
-13.315379518645521
4.385025239334158
9.29447505266955
11.615791427632912
20.42785869294143
16.177455559430403
21.97912275782919
